In [2]:
from pydantic_wrap import SimplePageFuncPydanticWithOutput, pydantic_model_from_type

In [5]:
def ff(x:int)->int:
    return x

mytype = ff.__annotations__["return"]

mod = pydantic_model_from_type(mytype)

In [10]:
mod.__fields__

{'field_name': ModelField(name='field_name', type=int, required=True)}